In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

print(bool(os.getenv("OPENAI_API_KEY")))

True


In [2]:
import PyPDF2

# Open the PDF file
with open('../data/coffee_processing.pdf', 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Get the number of pages
    num_pages = len(pdf_reader.pages)
    print(f"Total pages: {num_pages}")
    
    # Initialize variable to store all text
    all_text = []
    
    # Extract text from all pages
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        if text:  # Only add if page has text
            all_text.append(text.strip())
    
    # Combine all text with double newline between pages
    text_document = "\n\n".join(all_text)

    text_document = text_document.replace("Explore our developer-friendly HTML  to PDF API Printed using PDFCrowd HTML  to PDF", "")
    
    # Print the combined text
    print("\n=== COMBINED TEXT ===\n")
    print(text_document)
    
    # Optionally print stats
    print(f"\n\nTotal characters: {len(text_document)}")

Total pages: 4

=== COMBINED TEXT ===

Coffee Processing Methods
A Comprehensive Guide to Post-Harvest Coffee Processing
Coffee processing began in Ethiopia over 1,000 years ago.
Introduction to Coffee Processing
Coffee processing is the method used to transform freshly picked coffee cherries into green coffee
beans ready for roasting. The processing method significantly impacts the final flavor profile of the
coffee. Different regions around the world have developed unique processing techniques based on
their climate, available resources, and traditional practices. The choice of processing method can
enhance certain flavor characteristics while suppressing others, making it one of the most critical
decisions in coffee production.
Processing begins immediately after harvest, as coffee cherries are highly perishable. The outer
fruit must be removed, and the beans must be dried to prevent spoilage. The manner in which this
is accomplished varies widely, from ancient sun-drying methods to

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", ". "],
    chunk_size=300,
    chunk_overlap=40
)

chunks = text_splitter.create_documents([text_document])

for chunk in chunks:
    print(chunk.page_content.strip())
    print("-" * 20)
    print(len(chunk.page_content.strip()))
    print("-" * 20)

Coffee Processing Methods
A Comprehensive Guide to Post-Harvest Coffee Processing
Coffee processing began in Ethiopia over 1,000 years ago.
Introduction to Coffee Processing
Coffee processing is the method used to transform freshly picked coffee cherries into green coffee
beans ready for roasting
--------------------
297
--------------------
. The processing method significantly impacts the final flavor profile of the
coffee. Different regions around the world have developed unique processing techniques based on
their climate, available resources, and traditional practices
--------------------
235
--------------------
. The choice of processing method can
enhance certain flavor characteristics while suppressing others, making it one of the most critical
decisions in coffee production.
Processing begins immediately after harvest, as coffee cherries are highly perishable
--------------------
255
--------------------
. The outer
fruit must be removed, and the beans must be dried to preven

In [4]:
from langchain_openai import OpenAIEmbeddings
import chromadb

# Initialize embeddings model
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Set up ChromaDB client with persistent storage
chroma_client = chromadb.PersistentClient(path="../chroma_db")


In [6]:
collection = chroma_client.get_collection(name="my_documents_semantic_chunking")
print(collection.count())

0


In [5]:
# Delete existing collection if it exists (truncate)
try:
    chroma_client.delete_collection(name="my_documents_semantic_chunking")
    print("Existing collection deleted")
except:
    print("No existing collection found")

# Create new collection
collection = chroma_client.create_collection(
    name="my_documents_semantic_chunking",
    metadata={"description": "Document embeddings collection"}
)

No existing collection found


In [20]:
from langchain_experimental.text_splitter import SemanticChunker


splitter = SemanticChunker(
    embeddings=embeddings_model,
    breakpoint_threshold_type="percentile",  # or "standard_deviation"
    breakpoint_threshold_amount=95
)

chunks = splitter.split_text(text_document)

In [21]:
for chunk in chunks:
    print(chunk)
    print("-"*20)
    print(len(chunk))
    print("-"*20)


Coffee Processing Methods
A Comprehensive Guide to Post-Harvest Coffee Processing
Coffee processing began in Ethiopia over 1,000 years ago. Introduction to Coffee Processing
Coffee processing is the method used to transform freshly picked coffee cherries into green coffee
beans ready for roasting. The processing method significantly impacts the final flavor profile of the
coffee. Different regions around the world have developed unique processing techniques based on
their climate, available resources, and traditional practices. The choice of processing method can
enhance certain flavor characteristics while suppressing others, making it one of the most critical
decisions in coffee production. Processing begins immediately after harvest, as coffee cherries are highly perishable. The outer
fruit must be removed, and the beans must be dried to prevent spoilage. The manner in which this
is accomplished varies widely, from ancient sun-drying methods to modern mechanical processes. Each meth